In [97]:
import requests
import lxml.html
import pandas as pd
import sqlite3
from pandas.io import sql
import os
import time

In [98]:
def db_save(NEWS_LIST):
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con:
        try:
            NEWS_LIST.to_sql(name = 'NEWS_LIST', con = con, index = False, if_exists='append')
            #if_exists : {'fail', 'replace', 'append'} default : fail
        except Exception as e:
            print(str(e))
        print(len(NEWS_LIST), '건 저장완료..')

In [99]:
def db_delete():
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con:
        try:
            cur = con.cursor()
            sql = 'DELETE FROM NEWS_LIST'
            cur.execute(sql)
        except Exception as e:
            print(str(e))

In [100]:
def db_select():
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con:
        try:
            query = 'SELECT * FROM NEWS_LIST'
            NEWS_LIST = pd.read_sql(query, con = con)
        except Exception as e:
            print(str(e))
        return NEWS_LIST


In [101]:
import re
import string

def get_detail(url):
    body = []
    punc = '[!"#$%&\'()*+,-./:;<=>?[\]^_`{|}~“”·‘’'']'
    response = requests.get(url)
    root = lxml.html.fromstring(response.content)
    for p in root.xpath('//*[@class="article_view"]/section/p'):
        if p.text: # 체크
            body.append(re.sub(punc, '', p.text)) # 특수문자 제거
    full_body = ' '.join(body)

    return full_body

In [102]:
page = 70
max_page = 0
REG_DATE = '20230820'

In [103]:
while(True):
    df_list = []
    response = requests.get(f'http://news.daum.net/breakingnews/digital?page={page}&regDate={REG_DATE}')
    root = lxml.html.fromstring(response.content)
    for li in root.xpath('//*[@id="mArticle"]/div[3]/ul/li'):
        a = li.xpath('div/strong/a')[0]
        url = a.get('href')
        article = get_detail(url)
        print(url)
        print(article)
        print('-'*100)
        df_list.append([url, a.text, article])

    if df_list:
        df_10 = pd.DataFrame(df_list)
        df_10.columns = ['URL', 'TITLE', 'ARTICLE']
        db_save(df_10)

    # 페이지 번호 중에서 max 페이지 가져오기
    for a in root.xpath('//*[@id="mArticle"]/div[3]/div/span/a'):
        try:
            num = int(a.text)
            if max_page < num:
                max_page = num
        except:
            pass

    # 마지막 페이지 여부 확인
    span = root.xpath('//*[@id="mArticle"]/div[3]/div/span/a[@class="btn_page btn_next"]')

    if (len(span) <= 0) & (page >= max_page):
        break
    else:
        page = page + 1

    time.sleep(1)

None
None
None
None
None
None
None
None
None
None
None
None
https://v.daum.net/v/20230820061439968

----------------------------------------------------------------------------------------------------
https://v.daum.net/v/20230820060118890
생성형 인공지능AI은 우리가 일하는 방식은 물론이고 노는 방식에도 영향을 미칠 것입니다 하지만 기업 입장에서는 서둘러 생성형 AI를 사업 모델에 도입할 필요는 없다고 봅니다 기업은 생성형 AI를 어떤 용도로 어떻게 사용할지 활용하면 기대한 만큼의 이점을 얻을 수 있는지 고려한 후 도입해야 합니다 조선비즈가 다음달 21일 서울 소공동 웨스틴조선호텔에서 개최하는 국내 최대 테크 콘퍼런스 스마트클라우드쇼 2023Smart Cloud Show 2023의 첫번째 기조강연자로 무대에 서는 제리 카플란Jerry Kaplan 박사는 20일 서면 인터뷰를 통해 생성형 AI가 촉발하는 기회를 최대한 활용하기 위해서는 모든 과정을 재설계해야 가장 큰 이점을 얻을 수 있을 것이라며 이같이 말했다 인공지능의 미래 인간은 필요 없다 스타트업 실리콘밸리의 모험 저자인 카플란 박사는 35년 넘게 인공지능 전문가 연쇄창업가 미래학자 기술혁신가 등으로 활동하고 있다 그는 생성형 AI가 기업의 사업모델을 바꿔놓을지 묻자 그것은 인터넷이 기업의 사업모델을 어떻게 변화시키는지를 묻는 것과 같다고 했다 카플란 박사는 생성형 AI가 기업의 사업모델에 미칠 변화는 기업 산업 사업에 따라 다르다며 해당 질문에 대한 하나의 정답은 없으며 결국 각 기업이 최선의 답을 찾아 나서야 한다고 강조했다 카플란 박사는 1981년 AI 스타트업 테크놀리지Teknowledge를 공동 창업했다 1987년에는 태블릿 컴퓨터 회사 고GO Corp를 창업했다 카플란 박사는 애플이 아이패드를 선보인 것보다 

In [104]:
db_delete()

In [105]:
news_df = db_select()
print(news_df)

Empty DataFrame
Columns: [URL, TITLE, ARTICLE]
Index: []


In [106]:
excel = pd.ExcelWriter('news.xlsx')
news_df.to_excel(excel, 'Daum뉴스', index=False)
excel.save()

<ipython-input-106-298052c84b40>:3: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  excel.save()
